# Detecting EMA cross traps by using NEAT

### Import Library

In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
import seaborn as sns
import os

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import neat

### Load Price Data

In [3]:
import os
from pathlib import Path
notebook_path = os.getcwd()
current_dir = Path(notebook_path)
csv_file = str(current_dir.parent) + '/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print(csv_file)
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [4]:
data = dataset.copy()

In [5]:
data = data[data.index > '2020-11-01 00:00:00']

In [6]:
data["ema_fast"] = ta.ema(data["Close"], length=20)
data["ema_low"] = ta.ema(data["Close"], length=250)
data["ema_cross"] = ((data["ema_fast"] > data["ema_low"]) & (data["ema_fast"].shift(1) <= data["ema_low"].shift(1)) | (data["ema_fast"] < data["ema_low"]) & (data["ema_fast"].shift(1) >= data["ema_low"].shift(1)))

## Calculate some common features

In [7]:
data["ATR"] = ta.atr(data["High"], data["Low"], data["Close"], length=14)  # Volatility
data["RSI"] = ta.rsi(data["Close"], length=14)  # Momentum indicator

## TRAP labeling

In [8]:
def is_trap(r):
    trap = ''
    if r['ema_cross'] == True:
        if r['ema_fast'] > r['ema_low']:
            # Cross up
            if r['min_low_1dlater'] < r['Close'] - 3.5:
                trap = 1
            else:
                trap = 0
        else:
            # Cross down
            if r['max_high_1dlater'] > r['Close'] + 3.5:
                trap = 1
            else:
                trap = 0
    return trap

In [9]:
data['max_high_1dlater'] = data['High'].shift(-51).rolling(51).max()
data['min_low_1dlater'] = data['Low'].shift(-51).rolling(51).min()
data['trap'] = data.apply(lambda r: is_trap(r), axis=1)

In [10]:
# cross_data = data[data.ema_cross == True]
# len(cross_data[cross_data.trap == 0]) / len(cross_data['trap'])

## Features

In [11]:
data['RSI_s1'] = data['RSI'].shift(1)
data['RSI_s2'] = data['RSI'].shift(2)
data['RSI_s3'] = data['RSI'].shift(3)
data['RSI_s4'] = data['RSI'].shift(4)
data['RSI_s5'] = data['RSI'].shift(5)
data['RSI_s6'] = data['RSI'].shift(6)
data['RSI_s7'] = data['RSI'].shift(7)
data['RSI_s8'] = data['RSI'].shift(8)
cross_up_data = data[(data.ema_cross == True) & (data.ema_fast > data.ema_low)]
cross_up_data.dropna(inplace=True)

In [12]:
len(cross_up_data)

181

In [13]:
X = cross_up_data[['RSI', 'RSI_s1', 'RSI_s2', 'RSI_s3', 'RSI_s4', 'RSI_s5', 'RSI_s6', 'RSI_s7', 'RSI_s8', "trap"]]

# Train-Test Split
X_train, X_test = train_test_split(X, test_size=0.25, random_state=42)

In [14]:
len(X_train)

135

In [15]:
len(X_test)

46

In [16]:
def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        genome.fitness = 4.0
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        for move_index, row in X_train.iterrows():
            inputs = [row['RSI'], row['RSI_s1'], row['RSI_s2'], row['RSI_s3'], row['RSI_s4'], row["RSI_s5"], row["RSI_s6"], row["RSI_s7"], row["RSI_s8"]]
            expected_output = row['trap']
            output = net.activate(inputs)
            genome.fitness -= (output[0] - expected_output) ** 2


def run(config_file):
    # Load configuration.
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    # p.add_reporter(neat.StdOutReporter(True))
    # stats = neat.StatisticsReporter()
    # p.add_reporter(stats)

    # Run for up to 100 generations.
    winner = p.run(eval_genomes, 100)

    # Display the winning genome.
    print('\nBest genome:\n{!s}'.format(winner))
    return neat.nn.FeedForwardNetwork.create(winner, config)

In [17]:
%%time
config_path = os.path.join(current_dir, 'style-mix-1.cfg')
best_brain = run(config_path)


Best genome:
Key: 10414
Fitness: -24.970817679115434
Nodes:
	0 DefaultNodeGene(key=0, bias=0.1205060513603333, response=1.0, activation=sigmoid, aggregation=sum)
	939 DefaultNodeGene(key=939, bias=0.8393891655005974, response=1.0, activation=sigmoid, aggregation=sum)
	973 DefaultNodeGene(key=973, bias=-1.421042783088288, response=1.0, activation=sigmoid, aggregation=sum)
	1128 DefaultNodeGene(key=1128, bias=1.00236177285032, response=1.0, activation=sigmoid, aggregation=sum)
	1314 DefaultNodeGene(key=1314, bias=-1.1513211931905514, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-6, 939), weight=-1.2539586324620975, enabled=True)
	DefaultConnectionGene(key=(-6, 1314), weight=0.16682052526338798, enabled=True)
	DefaultConnectionGene(key=(-5, 939), weight=-2.60705246376809, enabled=True)
	DefaultConnectionGene(key=(-5, 973), weight=-0.8621781519572488, enabled=False)
	DefaultConnectionGene(key=(-5, 1314), weight=-1.3924920770678737, enabled=Fa

In [18]:
best_brain

In [19]:
# Show output of the most fit genome against training data.
outputs = []
for i, row in X_test.iterrows():
    inputs = [row['RSI'], row['RSI_s1'], row['RSI_s2'], row['RSI_s3'], row['RSI_s4'], row["RSI_s5"], row["RSI_s6"], row["RSI_s7"], row["RSI_s8"]]
    expected_output = row['trap']
    output = best_brain.activate(inputs)
    outputs.append(round(output[0]))
    # print("input {!r}, expected output {!r}, got {!r}".format(inputs, expected_output, output))

In [20]:
expected_outputs = X_test['trap'].to_list()
# Evaluate Performance
print("Accuracy:", accuracy_score(expected_outputs, outputs))

Accuracy: 0.717391304347826


In [21]:
X_test = X_test.assign(predict=outputs)

In [22]:
X_test

,RSI,RSI_s1,RSI_s2,RSI_s3,RSI_s4,RSI_s5,RSI_s6,RSI_s7,RSI_s8,trap,predict
Date,,,,,,,,,,,
2021-06-21 14:00:00,52.673689,52.509994,45.709590,48.698196,41.156158,47.861819,44.867536,47.779498,45.735577,1,1
2021-11-30 09:00:00,73.612596,63.045215,56.426958,57.081619,56.614215,54.785253,60.400612,53.940279,54.530333,1,1
2024-04-12 09:20:00,67.637243,75.089421,74.875214,73.160762,72.946487,62.336273,53.951266,53.444730,52.964293,1,1
2022-07-04 09:55:00,70.946635,74.425310,73.676235,76.161940,77.786556,76.131875,75.527318,74.319306,74.081709,1,1
2024-01-26 09:40:00,65.263963,64.685827,64.131485,63.600925,62.835224,61.060411,58.706084,59.447759,57.711410,0,1
2021-05-18 13:50:00,57.560489,46.756083,45.686103,47.457245,41.095860,41.289271,44.602945,41.796335,34.828180,1,1
2021-07-28 10:30:00,60.912946,60.251359,63.582444,61.402815,56.576173,50.871388,50.871388,53.343351,53.108259,1,1
2022-05-18 09:50:00,68.613205,62.026227,50.592734,51.656909,60.821011,61.572338,75.678794,75.678794,81.901534,1,1
2023-03-08 14:20:00,72.678563,68.329063,67.113700,63.998138,67.490595,66.645504,58.663013,69.015643,62.672142,0,1
